In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from optimum.quanto import QuantizedModelForCausalLM, qfloat8

# device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
tokenizer = AutoTokenizer.from_pretrained('THUDM/glm-4-9b-chat-hf')
model=QuantizedModelForCausalLM.from_pretrained("./models/bean980310__glm-4-9b-chat-hf_float8"
, device_map='auto')

message = [
    {
    "role": "system",
    "content": "Answer the following question."
    },
    {
        "role": "user",
        "content": "오늘 날씨는 어때?"
    }
]

inputs = tokenizer.apply_chat_template(
    conversation=message,
    return_tensors='pt',
    add_generation_prompt=True,
    return_dict=True,
).to(model.device)

input_len = inputs['input_ids'].shape[1]
generate_config = {
    "input_ids": inputs['input_ids'],
    "attention_mask": inputs['attention_mask'],
    "max_new_tokens": 128,
    "do_sample": False,
}
output = model.generate(**generate_config)
print(tokenizer.decode(output[0][input_len:], skip_special_tokens=True))

In [1]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model_name = "facebook/mbart-large-50-many-to-many-mmt"
tokenizer = MBart50TokenizerFast.from_pretrained(model_name)
model = MBartForConditionalGeneration.from_pretrained(model_name)

def translate(text, src_lang, tgt_lang):
    tokenizer.src_lang = src_lang
    encoded = tokenizer(text, return_tensors="pt")
    generated_tokens = model.generate(**encoded, forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang])
    translated = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    return translated[0]

tokenizer_config.json:   0%|          | 0.00/529 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/649 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/261 [00:00<?, ?B/s]

In [4]:
text="안녕하세요."

translated_text=translate(text, 'ko_KR', 'ja_XX')
translated_text

'こんにちは。'